# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [1]:
# Useful imports
import os
import hydra
import nest_asyncio
from IPython.core.display import display, HTML
from bokeh.io import output_notebook


/tmp/ipykernel_115641/4095267831.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

In [3]:
%cd ..

/media/sacardoz/Storage/nuplan-devkit


/home/sacardoz/miniconda3/envs/nuplan/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Simulating the planner <a name="simulation"></a>

## Prepare the simulation config

In [4]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(os.getenv('NUPLAN_TUTORIAL_PATH', ''), '../nuplan/planning/script')
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2.5h of 8 autolabeled logs in Las Vegas)
    'scenario_filter=one_continuous_log',  # simulate only one log
    "scenario_filter.log_names=['2021.06.14.16.48.02_veh-12_04057_04438']",
    'scenario_filter.limit_total_scenarios=1',  # use 1 total scenarios
]
ckpt_dir = '/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
hybrid_ckpt = '/home/sacardoz/checkpoints/pdm_offset_checkpoint.ckpt'
#'/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
#"/home/sacardoz/tutorial_vector_framework/training_simple_vector_experiment/train_default_simple_vector/2023.11.23.09.55.21/best_model/epoch.ckpt"
#"/home/sacardoz/training_raster_experiment/train_default_raster/2023.11.23.07.36.36/best_model/epoch.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
cfg = hydra.compose(config_name=simulation_hydra_paths.config_name, overrides=[
    '+simulation=closed_loop_multiagent',
    #'model=urban_driver_open_loop_model',
    'planner=pdm_hybrid_planner',
    f"planner.pdm_hybrid_planner.checkpoint_path={hybrid_ckpt}" ,
    #'observation.model_config=${model}',
    #f'observation.checkpoint_path={ckpt_dir}',
    f'observation.planner_type=pdm_hybrid',
    f'observation.pdm_hybrid_ckpt={hybrid_ckpt}',
    f'observation.occlusion_cfg.occlusion=true',
    f'observation.occlusion_cfg.manager_type=wedge',
    f'observation.optimization_cfg.mixed_agents=true',
    'worker=sequential',
    '+occlusion=true',
    '+occlusion.manager_type=wedge', #options: [range, shadow, wedge]
    "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
    *DATASET_PARAMS,
])


In [5]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import AGENT_TYPES, STATIC_OBJECT_TYPES, TrackedObjectType

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2024-01-09 19:40:42,731 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2024-01-09 19:40:42,732 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2024-01-09 19:40:42,732 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2024-01-09 19:40:42,732 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2024-01-09 19:40:42,732 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2024-01-09 19:40:42,732 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are st

In [6]:
runner = runners[0]

In [7]:
from nuplan.common.actor_state.agent import Agent
from nuplan.common.actor_state.oriented_box import OrientedBox
from nuplan.common.actor_state.scene_object import SceneObjectMetadata
from nuplan.common.actor_state.state_representation import StateSE2, StateVector2D

inserted_agent = Agent(tracked_object_type=TrackedObjectType.VEHICLE,
                       oriented_box=OrientedBox(StateSE2(664433, 3997400, -3.14 / 2), 5, 2, 2),
                       velocity=StateVector2D(0, 0),
                       metadata=SceneObjectMetadata(1623707858950113, "inserted", -2, "inserted"),
                       angular_velocity=0.0)

inserted_goal = StateSE2(664433, 3997000, -3.14 / 2)

In [8]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():

    if runner.simulation._time_controller.get_iteration().index == 10:
        runner.simulation._observations.add_agent_to_scene(inserted_agent, inserted_goal, iteration.time_point)

    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific call+back
    runner._simulation.callback.on_planner_start(runner.simulation.setup, runner.planner)

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(runner.simulation.setup, runner.planner, trajectory)

    iteration = runner.simulation._time_controller.get_iteration()
    print(iteration)
    runner.simulation.propagate(trajectory)
    # Execute specific callback
    runner.simulation.callback.on_step_end(runner.simulation.setup, runner.planner, runner.simulation.history.last())

runner.simulation.callback.on_simulation_end(runner.simulation.setup, runner.planner, runner.simulation.history)

SimulationIteration(time_point=TimePoint(time_us=1623707846350127), index=0)
SimulationIteration(time_point=TimePoint(time_us=1623707846450055), index=1)


KeyboardInterrupt: 

In [ ]:
from tutorials.utils.tutorial_utils import visualize_history
visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5003)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.3.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


2024-01-09 19:38:05,727 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet file size is 0 bytes
2024-01-09 19:38:05,728 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2024-01-09 19:38:05,728 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/nuboard/base/experiment_file_data.py:140}  Error creating dataset. Could not read schema from 'tutorials/media/nuplan_flow.svg'. Is this a 'parquet' file?: Could not open Parquet input source 'tutorials/media/nuplan_flow.svg': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
2024-01-09 19:38:05,729 INFO {/media/sacardoz/Storage/nuplan-devkit/nuplan/planning/nuboard/base/experiment_

Rendering a scenario: 100%|██████████| 1/1 [00:00<00:00,  7.39it/s]
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=1003&bokeh-absolute-url=http://localhost:5003&resources=none (::1) 1024.91ms
INFO:tornado.access:101 GET /ws?id=70a49c01-0a72-4cbb-8eb8-7b842945b222&origin=da2f890a-eb18-4637-9199-dd0f06169aef&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 1.11ms
INFO:bokeh.server.views.ws:ServerConnection created


2024-01-09 19:38:06,740 INFO {/home/sacardoz/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  200 GET /autoload.js?bokeh-autoload-element=1003&bokeh-absolute-url=http://localhost:5003&resources=none (::1) 1024.91ms
2024-01-09 19:38:06,740 INFO {/media/sacardoz/Storage/nuplan-devkit/tutorials/utils/tutorial_utils.py:267}  Done rendering!
2024-01-09 19:38:06,755 INFO {/home/sacardoz/miniconda3/envs/nuplan/lib/python3.9/site-packages/tornado/web.py:2344}  101 GET /ws?id=70a49c01-0a72-4cbb-8eb8-7b842945b222&origin=da2f890a-eb18-4637-9199-dd0f06169aef&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 1.11ms
